# Setup

In [1]:
!pip install requests dotenv pandas
from requests import get as fetch, post
from dotenv import load_dotenv
from os import environ
from enum import Enum
from pandas import DataFrame
from json import loads as read_json
from datetime import datetime

load_dotenv('.env')

is_production: bool = environ.get('environment') == 'production'
token: str = environ.get('production_token') if is_production else environ.get('sandbox_token')
url: str= environ.get('production_url') if is_production else environ.get('sandbox_url')

headers = {
    'Authorization': token,
    'accept': 'application/json'
}

if not token or not url: raise KeyError('Token or URL Not Found')

## Helpful Types

In [2]:
class TournamentIDs( Enum ):
    MLB = 109
    KBO_LEAGUE = 2541	
    PROFESSIONAL_BASEBALL = 1036
    NFL_PRESEASON = 233	
    NFL = 31
    NHL = 234	
    NBA = 132	

## Gather NBA Sport Events

In [10]:
mlb_data = fetch( f'{url}/affiliate/get_sport_events', headers=headers, params={ 'tournament_id': TournamentIDs.NBA.value } ).json()['data']['sport_events']

# parse data
nba_dataframe = DataFrame.from_records( mlb_data )

# filter todays gameeees
nba_todays_games = nba_dataframe[ nba_dataframe['scheduled'].str[:10] == datetime.now().strftime('%Y-%m-%d')] # only have to compare date
nba_todays_games['event_id'] = nba_todays_games['event_id'].apply( str )
nba_todays_event_ids = ','.join([ event_id for event_id in nba_todays_games['event_id'].to_list() ]) # parse to url friendly parameter

nba_todays_games = nba_todays_games.set_index('event_id')


/var/folders/rr/kn4kd8k57jbbxgg7rlwvfcx80000gn/T/ipykernel_55790/4230477943.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_todays_games['event_id'] = nba_todays_games['event_id'].apply( str )


## Gather Odds for each event

In [4]:

odd_data = fetch( f'{url}/v2/affiliate/get_multiple_markets', headers=headers, params={'event_ids':nba_todays_event_ids, 'get_all_markets':False}).json()['data']
odd_dataframe = DataFrame.from_dict( odd_data )

In [16]:
odd_dataframe

,20021985,20021986
0,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
1,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
2,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
3,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
4,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
5,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
6,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
7,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
8,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."
9,"{'category_name': 'Game Lines', 'display_name'...","{'category_name': 'Game Lines', 'display_name'..."


### Get Moneyline

In [13]:
for game in odd_dataframe.columns:
    moneyline = odd_dataframe[game][0]
    selections = moneyline['selections']
    odds = [ selection['odds'] for selection in selections if selection['odds']  ]
    line = max( odds ) if odds else 0
    nba_todays_games.loc[game, 'moneyline'] = line

In [15]:
nba_todays_games

,competitors,display_name,live_disabled,name,scheduled,sport_name,status,tournament_id,tournament_name,updated_at,moneyline
event_id,,,,,,,,,,,
20021985,"[{'abbreviation': 'IND', 'display_name': 'Indi...",Brooklyn Nets at Indiana Pacers,False,Brooklyn Nets at Indiana Pacers,2025-03-22T21:00:00Z,Basketball,not_started,132,NBA,1742574468765852000,100.0
20021986,"[{'abbreviation': 'ATL', 'display_name': 'Atla...",Golden State Warriors at Atlanta Hawks,False,Golden State Warriors at Atlanta Hawks,2025-03-22T23:00:00Z,Basketball,not_started,132,NBA,1742574468765852000,0.0


# Get favorite spread